In [3]:
import openai
import pandas as pd
import json
from dotenv import load_dotenv
import os
import time

In [4]:
load_dotenv()
openai.api_key = os.getenv('OPENAI_API_KEY')

# 사설 시설의 장점 도출

In [5]:
df = pd.read_csv('Data/cleaned_data_private.csv')

In [6]:
# 긴 리뷰 텍스트를 나누는 함수 정의
def split_text(text, max_tokens=3000):
    words = text.split()
    parts = []
    current_part = []
    current_length = 0

    for word in words:
        current_length += len(word) + 1  # 단어와 공백의 길이를 고려
        if current_length > max_tokens:
            parts.append(' '.join(current_part))
            current_part = [word]
            current_length = len(word) + 1
        else:
            current_part.append(word)

    if current_part:  # 마지막 부분 추가
        parts.append(' '.join(current_part))

    return parts

In [7]:
# 장점 도출 함수
def extract_pos_from_review(review_text):
    parts = split_text(review_text)
    all_issues = []

    for part in parts:
        prompt = f"""다음 블로그 글에서 시설에 대한 긍정적으로 평가한 점을 찾아줘:

        {part}

        만약 긍정적인 피드백이 명확하지 않거나 없으면 '없음'이라고 적고 끝내줘."""
        
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=240,  
            n=1,
            stop=None,
            temperature=0.5,
        )
        
        issues = response['choices'][0]['message']['content'].strip()
        
        if "없음" in issues or len(issues) < 10:  
            continue  
        else:
            all_issues.append(issues)
    
    if not all_issues:
        return "없음" # 글 나눴을 때 각 부분에 대해서 전부 없을 경우
    else:
        return '\n'.join(all_issues)

In [8]:
# api 호출시 rate limit에 걸릴 경우 재시도하는 함수
def extract_pos_with_retry(review_text, retries=3):
    for attempt in range(retries):
        try:
            return extract_pos_from_review(review_text)
        except openai.error.RateLimitError:
            print(f"Rate limit reached. Retrying in 20 seconds...")
            time.sleep(20)
        except openai.error.InvalidRequestError as e:
            print(f"Invalid request: {e}")
            return "Invalid request, please check the content length."
    return "Rate limit exceeded, please try later."

In [9]:
df.iloc[1]

location                                          천안 운동장 애견카페
title                             천안 애견카페 평생함께 잔디운동장과 포토존이 가득
url              https://blog.naver.com/nahe0904/223445138495
postdate                                           2024-05-13
content     ​ ​ 오늘 포스팅의 주인공은 제가 아닌 심바🦁에요 ​ 5월5일 개린이날에는 비가 ...
Name: 1, dtype: object

In [ ]:
# 실험용 **돌리면 토큰 씀
review_text = df.iloc[1].content
pos = extract_pos_from_review(review_text)
print(pos)

In [ ]:
# 전체 돌리기
# 장점 도출 및 저장
positive_points = {'천안 사설 시설': []}

for idx, row in df.iterrows():
    url = row['url']
    review_text = row['content']
    positives = extract_pos_with_retry(review_text)
    
    positive_points['천안 사설 시설'].append({
        'url': url,
        'review': review_text,
        'positives': positives
    })

In [13]:
positive_points['천안 사설 시설'][0]

{'url': 'https://blog.naver.com/kskr3105/223430968747',
 'review': '\u200b \u200b 오랜만에강아지운동장에서 뛰뛰시키고 싶어서천안애견카페날아라멍멍을 찾았어요. 주소상은 아산 탕정이지만 천안 불당동과 10분 거리밖에 되지 않아서 아주 가까웠어요. 직접 방문해 봤더니 블루 도어의 외관이 눈길을 사로잡았어요. 외국 감성이 뿜뿜하는 카페랄까요? 애카가 아니라 일반 갬성 카페 같은 분위기였어요. 바로 앞에넓은 단독 주차공간이 있어서 오시는 분들은 주차 걱정 안 해도 될 것 같아요. \u200b \u200b \u200b 알고 보니tv 동물농장에도 나왔던 곳이었어요. 아무튼 간에 입장하기 전 이용수칙이 있어서 열심히 읽어준 후 입장했어요. 우선, 반군이 애들을 데리고 강아지운동장으로 바로 입장했고 저 혼자만 실내로 들어갔어요. 주차장에서 운동장으로 들어가는 문이 있어서 편하게 바로 입장할 수 있었어요.날아라멍멍 영업시간은 평일은 오전 11시 ~ 오후 7시까지, 주말은 오후 8시까지이며 매주 월요일이 휴무이니 참고하셔서 방문하면 좋을 것 같아요. 저희는 낑깡이땜에 오픈 시간에 맞춰서 오전 11시에 방문했어요. \u200b \u200b \u200b 문을 열고 들어가니 안전문으로 한 번 더 있어서 실내 카페에서 놀다가 아이가 뛰쳐나가는 일은 없을듯해요. 그리고 실내화가 있어서 순간 신발을 벗는 건가 했는데 여쭤보니 비가 온 날에만 신는다고 하더라고요. 얼마나깔끔하게 관리를 하는지 이때부터 느낄 수 있었어요. \u200b \u200b \u200b 들어오자마자 저도 모르게 감탄이 절로 나왔어요. 사실 밖에서 봤을 땐 강아지운동장은 넓어도 실내는 아담한 천안애견카페라고 생각했어요. 근데약 50평 정도 되는 실내였으며 무엇보다 너무 깨끗했어요. 아무리 열자마자 왔다고 해도 제가 지금까지 직접 다녀봤던 애카들은 애들 냄새가 배어 있거나 바닥에 털들이 있고 그랬어요. 여기는 그런 거 전혀 없고 저희 집보다 쾌적하고 깔끔했어요. 

In [14]:
# 결과를 JSON 파일로 저장
with open('Data/private_facil_positives.json', 'w', encoding='utf-8') as json_file:
    json.dump(positive_points, json_file, ensure_ascii=False, indent=4)

print("Location positive points have been saved to private_facil_positives.json")

Location positive points have been saved to private_facil_positives.json


In [ ]:
with open('Data/private_facil_positives.json') as f:
    data = json.load(f)
data